In [9]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [10]:
import gym

env = gym.make("Taxi-v3",render_mode="ansi").env

env.reset(seed=0)

env = env.unwrapped

arr = env.render()
print(arr)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [11]:
print("Action Space ", env.action_space)
print("State Space ", env.observation_space)


Action Space  Discrete(6)
State Space  Discrete(500)


In [12]:
state = env.encode(4, 4, 3, 0)
print("State: ", state)

env.s = state

arr = env.render()
print(arr)


State:  492
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [13]:
env.P[env.s]


{0: [(1.0, 492, -1, False)],
 1: [(1.0, 392, -1, False)],
 2: [(1.0, 492, -1, False)],
 3: [(1.0, 472, -1, False)],
 4: [(1.0, 492, -10, False)],
 5: [(1.0, 492, -10, False)]}

In [14]:
state = env.encode(4, 4, 3, 0) # where the taxi is on the map

env.s = state

epochs = 0 # how many times he went around the map
penalties= 0 # how many errors in taking or leaving the clients in a wrong place

frames = [] # so we can see how he did it by frames

done = False # don't stop until you get to the target

while not done:
    action = env.action_space.sample() # if it is not done do the next step, .sample() so he can choose his actions randomly
    
    state, reward, done, _,_ = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    frames.append({
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    
    epochs += 1
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))


Timesteps taken: 1834
Penalties incurred: 610


In [15]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    actions = ['North', 'South', 'East', 'West', 'Pick-up', 'Drop-off']
    for i, frame in enumerate(frames):
        
        env.s = frame['state']
        
        clear_output(wait=True)
        
        arr = env.render()
        print(arr)
        
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {actions[frame['action']]}")
        print(f"Reward: {frame['reward']}")
        sleep(0.1)

print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1834
State: 0
Action: Drop-off
Reward: 20


In [16]:
episodes = 100

total_epochs = 0

total_penalties = 0

for _ in range(episodes):
    state = env.reset()
    
    penalties = 0
    reward = 0
    epochs = 0
    done = False
    
    while not done:
        action = env.action_space.sample()
        
        state, reward, done, _,_ = env.step(action)
        
        if reward == -10:
            penalties += 1
        
        epochs += 1
        
    total_penalties += penalties
    total_epochs += epochs
    
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {(total_epochs / episodes)}")
print(f"Average penalties per episode: {(total_penalties / penalties)}")


Results after 100 episodes:
Average timesteps per episode: 2350.0
Average penalties per episode: 282.3457249070632


In [18]:
import numpy as np

states = env.observation_space.n

actions = env.action_space.n

q_table = np.zeros([states, actions])
q_table


array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [19]:
alpha = 0.4
gamma = 0.6

episodes = 100000

for i in range(episodes):
    state = env.reset()[0]
    
    epochs = 0
    
    penalties = 0
    done = False
    
    while not done:
        action = np.argmax(q_table[state])

        old_value = q_table[state, action]

        next_state, reward, done, _,_ = env.step(action)
        next_state = next_state

        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max) # the equation

        q_table[state, action] = new_value

        state = next_state

print("Training finished.\n")


Training finished.



In [20]:
q_table


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-2.34417714, -2.34639386, -2.35182199, -2.34639386, -2.27325184,
        -4.        ],
       [-1.97237661, -1.97946063, -2.02409029, -1.97946063, -0.7504    ,
        -4.        ],
       ...,
       [-1.62175492, -1.66080164, -1.62175492, -1.67971024, -4.        ,
        -4.        ],
       [-2.19147423, -2.17358827, -2.19147423, -2.22884995, -4.        ,
        -4.        ],
       [-0.736     , -0.736     , -0.736     ,  1.28      , -4.        ,
        -4.        ]])

In [21]:
episodes = 100

total_epochs = 0

total_penalties = 0

for _ in range(episodes):
    state = env.reset()[0]
    
    penalties = 0
    reward = 0
    epochs = 0
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        
        state, reward, done, _,_ = env.step(action)
        
        if reward == -10:
            penalties += 1
        
        epochs += 1
    
    total_penalties += penalties
    
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {(total_epochs / episodes)}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Results after 100 episodes:
Average timesteps per episode: 13.28
Average penalties per episode: 0.0


In [27]:
state = env.encode(4, 4, 3, 0)
print("State:", state)

env.s = state

arr=env.render()
print(arr)

frames = []

done = False

while not done:
    action = np.argmax(q_table[state])
    
    state, reward, done, _, _ = env.step(action)
    
    frames.append({
        'state': state,
        'action': action,
        'reward': reward
        }
    )

print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 10
State: 0
Action: Drop-off
Reward: 20
